#**Pré-processamento**

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd
import seaborn as sns
import missingno as msno
import warnings
from matplotlib import pyplot as plt
import datetime
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)

df = pd.read_csv("gdrive/My Drive/dissertação/Qualidade_do_ar_-_Dados_horarios.csv",sep=",")
df['Data']=df['Data'].astype('str')
df['Data']=df['Data'].apply(lambda x: x[:-3])

#conversão para datetime
def convert_to_datetime(x):
    return(datetime.datetime.strptime(x, "%Y/%m/%d %H:%M:%S"))
df['Data']=df['Data'].apply(convert_to_datetime) 
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-8:-3])
# #extrair ano, mes, hora 
df['Ano']=df['Data'].apply(lambda x: x.year)
df['Mês']=df['Data'].apply(lambda x: x.month)
df['Dia']=df['Data'].apply(lambda x: x.day)
df['Hora']=df['Data'].apply(lambda x: x.hour)
df['Hora-minuto']=df['Data'].apply(lambda x: str(x)[-11:-6])

df_bg=df[df['Estação']=='BG'] #Bangu
df_gc=df[df['Estação']=='CG'] #Campo Grande
df_ca=df[df['Estação']=='CA'] #Centro
df_av=df[df['Estação']=='AV'] #Copacabana
df_ir=df[df['Estação']=='IR'] #Irajá
df_pg=df[df['Estação']=='PG'] #Pedra de Guaratiba
df_sc=df[df['Estação']=='SC'] #São Cristóvão
df_sp=df[df['Estação']=='SP'] #Tijuca

df_sc.set_index('Data',inplace=True)
df_sc.drop(columns=['OBJECTID','CodNum','Estação','Dir_Vento','NO2','HCNM','HCT','CH4','NO','NOx',\
                   'PM2_5', 'Lat', 'Lon', 'X_UTM_Sirgas2000','Y_UTM_Sirgas2000'], inplace=True)
df_sc_2=df_sc.copy() #para fazer distribuição sem eliminar ausentes

df_sc=df_sc['2011-01-01':'2018-01-01']
#explicar que usou interpolação linear com spline e justificar
df_sc['Chuva'].interpolate(method='slinear', inplace=True)
df_sc['Pres'].interpolate(method='slinear', inplace=True)
df_sc['RS'].interpolate(method='slinear', inplace=True)
df_sc['Temp'].interpolate(method='slinear', inplace=True)
df_sc['UR'].interpolate(method='slinear', inplace=True)
df_sc['Vel_Vento'].interpolate(method='slinear', inplace=True)
df_sc['SO2'].interpolate(method='slinear', inplace=True)
df_sc['CO'].interpolate(method='slinear', inplace=True)
df_sc['O3'].interpolate(method='slinear', inplace=True)
df_sc['PM10'].interpolate(method='slinear', inplace=True)

Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#**Gridsearch**

In [3]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import LSTM
from sklearn.metrics import mean_squared_error #para obter rmse usar formula
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam
from keras.constraints import maxnorm
from keras import metrics
from sklearn.metrics import make_scorer

In [4]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score #colocar citação

from math import sqrt
from numpy import concatenate
from matplotlib import pyplot

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import Adam

df_sc.drop(columns=['Ano','Mês','Dia','Hora','Hora-minuto'], inplace=True)
#df_sc.drop(columns=['Hora-minuto'], inplace=True)
df_sc_copy=df_sc.copy()
df_sc_copy=df_sc_copy[['O3']]
df_sc.drop(columns=['O3'],inplace=True)
df_sc = pd.concat([df_sc_copy,df_sc], axis=1, ignore_index=False)

values = df_sc.values
# ensure all data is float
values = values.astype('float32')
# normalize features

# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled = scaler.fit_transform(values)

scaled=values #não normalizado

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# specify the number of lag hours
n_hours = 4 #aqui definimos o LAg
n_features = 10

# frame as supervised learning
# reframed = series_to_supervised(scaled, n_hours, 1)
reframed = series_to_supervised(scaled, n_hours, 4) #é aqui que definimos a janela de previsão

# split into train and test sets
values = reframed.values
n_train_hours = 1855 * 24
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]

# split into input and outputs
n_obs = n_hours * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, n_features))

# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.0005,momentum=0.4,neurons=1):
    # create model
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    optimizer = Adam(lr=learn_rate)
    # Compile model
    model.compile(loss='mae', optimizer=optimizer, metrics=[metrics.mean_squared_error,metrics.mean_absolute_error,metrics.mean_absolute_percentage_error])
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

model = KerasRegressor(build_fn=create_model, verbose=0)
scoring = {'mse': make_scorer(mean_squared_error),'mae':make_scorer(mean_absolute_error),'r2':make_scorer(r2_score)}
learn_rate=[0.0005]
neurons = [100,200,300]
batch_size = [500]
epochs = [150,200,250,300]

param_grid = dict(neurons=neurons, epochs=epochs,learn_rate=learn_rate,batch_size=batch_size)

grid = GridSearchCV(estimator=model, param_grid=param_grid,scoring=scoring,refit='mse',return_train_score=True,n_jobs=-1, cv=3)

grid_result = grid.fit(train_X, train_y)

In [5]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 312.789785 using {'batch_size': 500, 'epochs': 150, 'learn_rate': 0.0005, 'neurons': 100}


In [6]:
grid_results=pd.DataFrame(grid_result.cv_results_)

In [7]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [8]:
grid_results.to_csv('data.csv')
!cp data.csv "drive/My Drive/"

In [9]:
grid_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_epochs,param_learn_rate,param_neurons,params,split0_test_mse,split1_test_mse,split2_test_mse,mean_test_mse,std_test_mse,rank_test_mse,split0_train_mse,split1_train_mse,split2_train_mse,mean_train_mse,std_train_mse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,split0_train_mae,split1_train_mae,split2_train_mae,mean_train_mae,std_train_mae,split0_test_r2,split1_test_r2,split2_test_r2,mean_test_r2,std_test_r2,rank_test_r2,split0_train_r2,split1_train_r2,split2_train_r2,mean_train_r2,std_train_r2
0,55.689452,4.443695,0.524574,0.015857,500,150,0.0005,100,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",322.527435,219.125946,396.715973,312.789785,72.827059,1,263.402649,326.982971,273.900146,288.095256,27.829723,11.226665,10.124678,12.854399,11.401914,1.121273,1,10.399300,11.489376,10.267324,10.718667,0.547631,0.522006,0.511681,0.426172,0.486620,0.042951,12,0.543226,0.522584,0.513682,0.526498,0.012375
1,48.805754,0.991234,0.556718,0.036586,500,150,0.0005,200,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",278.891479,195.675186,325.742462,266.769709,53.787093,6,222.923477,276.215271,229.595688,242.911479,23.706350,10.663934,9.199282,11.762749,10.541988,1.050077,6,9.707415,10.404185,9.454308,9.855302,0.401638,0.586676,0.563940,0.528831,0.559816,0.023795,7,0.613422,0.596708,0.592346,0.600826,0.009083
2,54.455655,4.179348,0.588222,0.019057,500,150,0.0005,300,"{'batch_size': 500, 'epochs': 150, 'learn_rate...",280.822906,197.998550,334.607086,271.142848,56.188674,5,223.116165,240.610550,227.650253,230.458989,7.413057,10.418627,9.484260,11.795218,10.566035,0.949185,5,9.439117,9.844497,9.300479,9.528031,0.230822,0.583814,0.558763,0.516009,0.552862,0.027994,8,0.613088,0.648693,0.595800,0.619194,0.022021
3,63.590842,1.493691,0.617998,0.017087,500,200,0.0005,100,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",310.604034,210.362183,359.652985,293.539734,62.130662,3,257.646027,312.875885,247.471344,272.664419,28.735606,11.105074,9.918762,12.175835,11.066557,0.921848,3,10.289479,11.267863,9.902417,10.486587,0.574601,0.539677,0.531211,0.479781,0.516890,0.026466,10,0.553209,0.543181,0.560608,0.552333,0.007141
4,65.563679,1.487413,0.612947,0.039839,500,200,0.0005,200,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",273.414276,204.119812,315.383392,264.305827,45.877511,8,215.215836,260.701935,221.782745,232.566839,20.074342,10.694710,9.554859,11.335649,10.528406,0.736453,7,9.576716,10.210066,9.248006,9.678263,0.399269,0.594793,0.545122,0.543815,0.561243,0.023730,5,0.626788,0.619359,0.606218,0.617455,0.008505
5,72.340662,2.651447,0.578239,0.047039,500,200,0.0005,300,"{'batch_size': 500, 'epochs': 200, 'learn_rate...",275.109924,194.219284,297.652710,255.660639,44.409642,10,213.036453,239.011017,208.778870,220.275447,13.361585,10.384458,9.252815,11.051219,10.229497,0.742327,11,9.217208,9.761138,9.023890,9.334078,0.312119,0.592280,0.567185,0.569461,0.576309,0.011332,3,0.630568,0.651029,0.629307,0.636968,0.009956
6,78.220257,1.310695,0.568786,0.039177,500,250,0.0005,100,"{'batch_size': 500, 'epochs': 250, 'learn_rate...",319.329041,222.596634,365.842712,302.589462,59.665847,2,258.066498,331.270966,253.755508,281.030991,35.568595,11.073848,10.270345,12.419828,11.254674,0.886790,2,10.195298,11.623379,10.004618,10.607765,0.722354,0.526747,0.503946,0.470828,0.500507,0.022958,11,0.552480,0.516323,0.549450,0.539418,0.016377
7,79.838920,2.087071,0.560638,0.033051,500,250,0.0005,200,"{'batch_size': 500, 'epochs': 250, 'learn_rate...",276.355194,193.652817,328.863281,266.290431,55.656338,7,213.283707,242.995514,217.346085,224.541768,13.153739,10.519538,9.256397,11.619526,10.465154,0.965509,8,9.339199,9.822780,9.132471,9.431483,0.289274,0.590435,0.568447,0.524317,0.561066,0.027492,6,0.630139,0.645211,0.614096,0.629815,0.012705
8,89.792014,7.090309,0.640040,0.015919,500,250,0.0005,300,"{'batch_size': 500, 'epochs': 250, 'learn_r